In [1]:
# !pip install pandas
# !pip install requests
# !pip install bs4
# !pip install plotly

In [2]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

URL = "https://www.reclameaqui.com.br/empresa/norton/lista-reclamacoes/?pagina=31"

# URL = "https://www.reclameaqui.com.br/empresa/norton/lista-reclamacoes/?pagina=1&status=EVALUATED"
# URL = "https://www.reclameaqui.com.br/empresa/norton/lista-reclamacoes/?pagina=2&status=EVALUATED"
# URL = "https://www.reclameaqui.com.br/empresa/norton/lista-reclamacoes/?pagina=3&status=EVALUATED"

html_data = requests.get(URL, headers=HEADERS)

In [4]:
beautiful_soup = BeautifulSoup(html_data.content, 'html.parser')
# print(beautiful_soup.prettify())

In [5]:
beautiful_soup.title





<title>Todas as reclamações  - Norton - Reclame Aqui</title>

In [6]:
linksList = []

dataLink = beautiful_soup.findAll('div',attrs={'class':'sc-1pe7b5t-0 bJdtis'})
# dataTitle = beautiful_soup.findAll('h4',attrs={'class':'sc-1pe7b5t-1 fTrwHU'})

for div in dataLink:
    links = div.findAll('a')
    dataTitle = div.find('h4').getText()
    print(dataTitle)
    for a in links:
        # print(a['href'] )
        # print(dataTitle['title'] )

        linksList.append(a['href'])

print(len(linksList))

Falha na tentativa de reembolso 
COBRANÇA RENOVAÇÃO ABUSIVA
Cobrança indevida
Cobrança indevida
Dificuldade para cancelar renovação automatica
Cobrança indevida 
Cobrança indevida
Renovação
Pagamento indevido 
norton antivirus debitou automatico da minha conta e nao extorna o valor
10


In [12]:
# dataframe from our list

RA_df = pd.DataFrame(columns=["CaseID", "Date", "Time", "LocationCity", "LocationState", "Status", "dealAgain", "Rate", "Links"])

for i  in range(len(linksList)):

    intoURL = "https://www.reclameaqui.com.br" + linksList[i]

    html_data_RA = requests.get(intoURL, headers=HEADERS)

    beautiful_soup_RA = BeautifulSoup(html_data_RA.content, 'html.parser')
    # print(beautiful_soup_RA.title)

    # data_RA = beautiful_soup_RA.findAll('div')

    # complaintID = beautiful_soup_RA.find('div',attrs={'data-testid':'complaint-id'}).getText()
    # complaintIDnumber = complaintID.split()
    #
    # complaintCreationDate = beautiful_soup_RA.find('div',attrs={'class':'lzlu7c-6 lzlu7c-8 dLWUvV ePqLRr'}).getText()
    # complaintCreation = complaintCreationDate.split()
    #
    # complaintLocation = beautiful_soup_RA.find('div',attrs={'class':'lzlu7c-6 lzlu7c-7 dLWUvV erEgDU'}).getText()
    # complaintLocation = complaintLocation.split("-")

    complaint = beautiful_soup_RA.find('div',attrs={'class':'lzlu7c-10 fYCrvq'})
    # print(complaint)
    complaintID = complaint.find('span',attrs={'data-testid':'complaint-id'}).getText()
    complaintIDnumber = (complaintID.split())
    complaintCreationDate = complaint.find('span',attrs={'data-testid':'complaint-creation-date'}).getText()
    complaintCreation = (complaintCreationDate.split())
    complaintLocation = complaint.find('span',attrs={'data-testid':'complaint-location'}).getText()
    complaintLocation = (complaintLocation.split("-"))

    complaintStatus = beautiful_soup_RA.find('div',attrs={'data-testid':'complaint-status'}).getText()

    if complaintStatus == "Respondida" or "Em réplica":
        dealAgainStatus = "NaN"
        valueRate = "NaN"
    if complaintStatus == "Resolvido" :
        dealAgainStatus = beautiful_soup_RA.find('div',attrs={'data-testid':'complaint-deal-again'}).getText()
        valueRate = beautiful_soup_RA.find('div',attrs={'class':'uh4o7z-3 ceUcTc'}).getText()


    RA_df = RA_df.append({"CaseID":complaintIDnumber[1],
                          "Date":complaintCreation[0],
                          "Time":complaintCreation[2],
                          "LocationCity":complaintLocation[0],
                          "LocationState":complaintLocation[1],
                          "Status":complaintStatus,
                          "dealAgain":dealAgainStatus,
                          "Rate":valueRate,
                          "Links":linksList[i]}, ignore_index=True)



C:\Users\ediva\AppData\Local\Temp\ipykernel_18156\672629917.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RA_df = RA_df.append({"CaseID":complaintIDnumber[1],
C:\Users\ediva\AppData\Local\Temp\ipykernel_18156\672629917.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RA_df = RA_df.append({"CaseID":complaintIDnumber[1],
C:\Users\ediva\AppData\Local\Temp\ipykernel_18156\672629917.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RA_df = RA_df.append({"CaseID":complaintIDnumber[1],
C:\Users\ediva\AppData\Local\Temp\ipykernel_18156\672629917.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RA_df = RA_df.append({"CaseID":co

In [13]:
RA_df.head(10)


,CaseID,Date,Time,LocationCity,LocationState,Status,dealAgain,Rate,Links
0,148212687,09/08/2022,17:36,Rio de Janeiro,RJ,Em réplica,NaN,NaN,/norton/falha-na-tentativa-de-reembolso_O5eMCk...
1,148191909,09/08/2022,13:23,São Paulo,SP,Resolvido,Sim,9,/norton/cobranca-renovacao-abusiva_fLjhBkZIyNd...
2,148165221,09/08/2022,08:03,São Paulo,SP,Respondida,NaN,NaN,/norton/cobranca-indevida_ccUq2UxK7sqwfD5E
3,148146293,08/08/2022,19:30,Belo Horizonte,MG,Resolvido,Sim,10,/norton/cobranca-indevida_SqrrNpVvHTcrhtV8
4,148142421,08/08/2022,18:33,São José dos Campos,SP,Respondida,NaN,NaN,/norton/dificuldade-para-cancelar-renovacao-au...
5,148126479,08/08/2022,15:15,Itupeva,SP,Em réplica,NaN,NaN,/norton/cobranca-indevida_d84k3hh2Wm90QPVH
6,148121105,08/08/2022,14:14,Bragança Paulista,SP,Resolvido,Sim,8,/norton/cobranca-indevida_BC6i6wdTjCfVU5Cp
7,148097049,08/08/2022,09:45,Campinas,SP,Respondida,NaN,NaN,/norton/renovacao_KSgBiKslBFyJO0R4
8,148088787,08/08/2022,00:45,Florianópolis,SC,Respondida,NaN,NaN,/norton/pagamento-indevido_23ygBFHhO92RNl80
9,148066865,07/08/2022,10:30,Porto Alegre,RS,Resolvido,Sim,8,/norton/norton-antivirus-debitou-automatico-da...


In [9]:
def getLinks(urlLink):
    html_data = requests.get(urlLink, headers=HEADERS)
    beautiful_soup = BeautifulSoup(html_data.content, 'html.parser')
    linksList = []
    dataLink = beautiful_soup.findAll('div',attrs={'class':'sc-1pe7b5t-0 bJdtis'})
    for div in dataLink:
        links = div.findAll('a')
        dataTitle = div.find('h4').getText()
        print(dataTitle)
        for a in links:
            print(a['href'] )
            linksList.append(a['href'])


In [10]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

URL = "https://www.reclameaqui.com.br/empresa/norton/lista-reclamacoes/?pagina=31"

print( getLinks(URL) )

Falha na tentativa de reembolso 
/norton/falha-na-tentativa-de-reembolso_O5eMCkI5eGUZghuc
COBRANÇA RENOVAÇÃO ABUSIVA
/norton/cobranca-renovacao-abusiva_fLjhBkZIyNdOBsxF
Cobrança indevida
/norton/cobranca-indevida_ccUq2UxK7sqwfD5E
Cobrança indevida
/norton/cobranca-indevida_SqrrNpVvHTcrhtV8
Dificuldade para cancelar renovação automatica
/norton/dificuldade-para-cancelar-renovacao-automatica_bPTJdrsq54F5qOob
Cobrança indevida 
/norton/cobranca-indevida_d84k3hh2Wm90QPVH
Cobrança indevida
/norton/cobranca-indevida_BC6i6wdTjCfVU5Cp
Renovação
/norton/renovacao_KSgBiKslBFyJO0R4
Pagamento indevido 
/norton/pagamento-indevido_23ygBFHhO92RNl80
norton antivirus debitou automatico da minha conta e nao extorna o valor
/norton/norton-antivirus-debitou-automatico-da-minha-conta-e-nao-extorna-o-valor_khj3YqbKDtzdrCfc
None


In [11]:
print(complaint)
print(re.split('-| ', complaint))

<div class="lzlu7c-10 fYCrvq"><div class="lzlu7c-6 lzlu7c-7 dLWUvV erEgDU"><svg aria-hidden="true" class="MuiSvgIcon-root" focusable="false" viewbox="0 0 24 24"><path d="M12,11.5A2.5,2.5 0 0,1 9.5,9A2.5,2.5 0 0,1 12,6.5A2.5,2.5 0 0,1 14.5,9A2.5,2.5 0 0,1 12,11.5M12,2A7,7 0 0,0 5,9C5,14.25 12,22 12,22C12,22 19,14.25 19,9A7,7 0 0,0 12,2Z"></path></svg><span data-testid="complaint-location">Porto Alegre - RS</span></div><div class="lzlu7c-6 lzlu7c-8 dLWUvV ePqLRr"><svg aria-hidden="true" class="MuiSvgIcon-root" focusable="false" viewbox="0 0 24 24"><path d="M19,19H5V8H19M16,1V3H8V1H6V3H5C3.89,3 3,3.89 3,5V19A2,2 0 0,0 5,21H19A2,2 0 0,0 21,19V5C21,3.89 20.1,3 19,3H18V1"></path></svg><span data-testid="complaint-creation-date">07/08/2022 às 10:30</span></div><div class="lzlu7c-6 lzlu7c-9 dLWUvV fuXXZq"><span class="lzlu7c-12 joySdk" data-testid="complaint-id"><b>ID:</b> 148066865</span></div></div>


TypeError: expected string or bytes-like object